# Segmentating and Clustering Neighborhoods in Toronto (Part 1)

** 1st thing to do is to import all necessary libraries **

In [37]:
import numpy as np
import pandas as pd
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import requests
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
print("Imported libraries")

Imported libraries


** 2nd: scrap data from Wikipedia into a dataframe **

In [40]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(data, 'html.parser')


In [47]:
column_names = ['Postalcode','Borough','Neighborhood']

toronto = pd.DataFrame(columns = column_names)

content = soup.find('div',class_ = 'mw-parser-output')

table = content.table.tbody

postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i += 1
        elif i == 1:
            borough = td.text
            i += 1
        elif i == 2:
            neighborhood = td.text.strip('\n').replace(']','')

    toronto = toronto.append({'Postalcode':postcode, 'Borough':borough, 'Neighborhood':neighborhood},ignore_index = True)



In [51]:
toronto = toronto[toronto.Borough != 'Not assigned']
toronto = toronto[toronto.Borough != 0]

toronto.reset_index(drop = True, inplace = True)


for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i += 1

In [53]:
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(','.join).reset_index()

In [54]:
df

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn
...,...,...,...
175,M9V\n,Etobicoke\n,"South Steeles, Silverstone, Humbergate, Jamest..."
176,M9W\n,Etobicoke\n,"Northwest, West Humber - Clairville"
177,M9X\n,Not assigned\n,Not assigned\n
178,M9Y\n,Not assigned\n,Not assigned\n


In [55]:
df.shape

(180, 3)